# Documention Agent

This notebook demonstrates a documention agent:
1. Video-to-protocol conversion using Gemini API
2. Gradio web interface for documention agent

In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

from IPython.display import Markdown

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import documentation_agent as doc_agent

In [2]:
# !pip install -q -U google-generativeai
# !pip install gradio

In [3]:
import configparser

import google.generativeai as genai

config = configparser.ConfigParser()
config.read("../secrets.ini")

api_key = config["DEFAULT"]["API_KEY"]

genai.configure(api_key=api_key)

In [4]:
# Test if the API Key is working

# model = genai.GenerativeModel("gemini-1.5-flash")  # Use the Gemini model

# response = model.generate_content("Hello, how are you?")  # Send a prompt

# print(response.text)  # Print the model's response

In [5]:
# Define the prompts
prompt_1 = "Example Input video: "
prompt_2 = "Example Output protocol:"
prompt_3 = "Glossary:"
prompt_4 = """
You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry. You are known for your methodological rigorand clear scientific writing, particularly in the style of Nature Protocols.
Your task is to analyze the provided video and extract a detailed experimental protocol.
You always stick to the facts in the sources provided, and never make up new facts.
You use the terminology provided in the glossary accurately throughout your response.
Take a deep breath and think step by step.
Answer direct.
"""
prompt_5 = "Input Video:"
prompt_6 = "Output Protocol:"

In [9]:
few_shot_example_folder = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/"

# Define the few shot example and domain knowldge
video_file_name = Path(few_shot_example_folder, "tims_calibration.mp4")
video_example = doc_agent.upload_video_and_wait(video_path=video_file_name)

path_example = Path(few_shot_example_folder, "tims_calibration.md")
file_example = doc_agent.read_and_encode_document(path_example)

path_glossary = Path.resolve(Path(os.pardir, "data", "glossary.md"))
glossary = doc_agent.read_and_encode_document(path_glossary)

config = doc_agent.ProcessingConfig(
    prompts=[prompt_1, prompt_2, prompt_3, prompt_4, prompt_5, prompt_6],
    video_examples=[video_example],
    file_examples=[file_example],
    glossary=glossary,
)

In [21]:
video_path = (
    "/Users/patriciaskowronek/Documents/LlmProjectIdeas/Documentation/Trimmed_video.mov"
)

# Process video with caching
# cache.delete()
# output, filename = doc_agent.process_video(video_path, config)

# Process video without caching
output, filename = doc_agent.process_video(video_path, config, cache_strategy=False)

doc_agent.generate_markdown_for_download(output, filename)

print(filename)
Markdown(output)

2025-03-06 11:08:18,366 - documentation_agent - INFO - Uploading file: /Users/patriciaskowronek/Documents/LlmProjectIdeas/Documentation/Trimmed_video.mov
2025-03-06 11:08:20,542 - documentation_agent - INFO - Upload completed. File URI: https://generativelanguage.googleapis.com/v1beta/files/f0jjp39fx4ho
2025-03-06 11:08:30,729 - documentation_agent - INFO - Video processing completed successfully


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/patriciaskowronek/Conda/miniconda3/envs/docu_test/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/54/g1_1ycl12hl02xj_g_nm_6cm0000gn/T/ipykernel_84976/519042334.py", line 10, in <module>
    output, filename = doc_agent.process_video(video_path, config, cache_strategy=False)
  File "/Users/patriciaskowronek/Documents/proteomics_specialist/proteomics_specialist/documentation_agent.py", line 215, in process_video
    return _process_without_cache(processed_video, config, video_file)
  File "/Users/patriciaskowronek/Documents/proteomics_specialist/proteomics_specialist/documentation_agent.py", line 368, in _process_without_cache
    return _generate_response(
  File "/Users/patriciaskowronek/Documents/proteomics_specialist/proteomics_specialist/documentation_agent.py", line 305, in _generate_response
    response = model.generate

In [9]:
import documentation_agent as doc_agent
import gradio as gr

css = """
#top-title {font-size: 2rem; text-align: center; margin-bottom: 1rem;}
.download-row {margin-top: 1rem; padding: 1rem; border-top: 1px solid #eee;}
"""

with gr.Blocks(css=css) as iface:
    gr.HTML("""
        <h1 id="top-title">Documentation agent</h1>
    """)

    with gr.Row():
        video_input = gr.Video(label="Upload Video")

    gr.Markdown("# Protocol extracted from Video:")

    with gr.Row():
        text_output = gr.Markdown(label="Output Text")
        filename_state = gr.State()

    with gr.Row(elem_classes="download-row"):
        with gr.Column(scale=2):
            download_button = gr.Button(
                "Generate Protocol", variant="primary", size="lg"
            )
        with gr.Column(scale=3):
            file_output = gr.File(
                label="Download Protocol", file_count="single", type="filepath"
            )

    video_input.change(
        fn=lambda x: doc_agent.process_video(x, config),
        inputs=video_input,
        outputs=[text_output, filename_state],
    )

    download_button.click(
        fn=doc_agent.generate_markdown_for_download,
        inputs=[text_output, filename_state],
        outputs=file_output,
    )

iface.launch()

2025-01-28 18:22:19,356 - httpx - INFO - HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"
2025-01-28 18:22:19,360 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7860/startup-events "HTTP/1.1 200 OK"
2025-01-28 18:22:19,370 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


2025-01-28 18:22:19,494 - httpx - INFO - HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
2025-01-28 18:22:19,507 - httpx - INFO - HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
2025-01-28 18:22:20,114 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-01-28 18:22:26,364 - documentation_agent - INFO - Uploading file: /private/var/folders/54/g1_1ycl12hl02xj_g_nm_6cm0000gn/T/gradio/570716d2985435122e0b104197f76b587c380ff40548f995f2f058999d9079e5/Trimmed_video.mov
2025-01-28 18:22:28,433 - documentation_agent - INFO - Upload completed. File URI: https://generativelanguage.googleapis.com/v1beta/files/96s95c2btm8a
2025-01-28 18:22:38,632 - documentation_agent - INFO - Video processing completed successfully
2025-01-28 18:22:55,670 - documentation_agent - INFO - Model usage metadata: prompt_token_count: 57056
candidates_token_count: 2174
total_token_count: 59230
cached_content_token_count: 52332

